# 第六章

os.chdir("D:\\TSBOOKDATA\\Chap6") #改变工作目录   

### 图6.1

ar = np.r_[1, 0,0,0,0,0,0,0,0,0,0,0,-0.7]  
ma1 = np.r_[1, 0.4]; ma2= np.r_[1, -0.4]  
np.random.seed(601)  
mod_data1 = sm.tsa.arma_generate_sample(ar, ma1, 200)  
np.random.seed(602)  
mod_data2 = sm.tsa.arma_generate_sample(ar, ma2, 200)  
mod_Series1 = pd.Series(mod_data1)  
mod_Series2 = pd.Series(mod_data2)  

fig = plt.figure(figsize=(12,4),dpi=150)  
ax1 = fig.add_subplot(121)  
ACF(mod_Series1,lag=60); plt.legend(['$θ=0.4, ϕ=0.7$'])  
ax2 = fig.add_subplot(122)  
ACF(mod_Series2,lag=60, fname='fig/6_1.png')  
plt.legend(['$θ=-0.4, ϕ=0.7$'])  

### 例6.1

carb_data = np.loadtxt("carbondioxide.txt")  
index = pd.date_range(start="1966-01", end="1991-01", freq="M")  
carb_df = pd.Series(carb_data,index=index)  
carb_diff = carb_df.diff()  

fig = plt.figure(figsize=(12,6),dpi=150)  
ax1 = fig.add_subplot(211)  
ax1.plot(carb_df, linestyle="-", color="blue")  
ax1.set_ylabel(ylabel="二氧化碳排放量",fontsize=17)  
ax1.set_xlabel(xlabel="时间", fontsize=17)  
plt.xticks(fontsize=15); plt.yticks(fontsize=15)  
ax2 = fig.add_subplot(212)  
ax2.plot(carb_diff, linestyle="-", color="blue")  
ax2.set_ylabel(ylabel="一阶差分", fontsize=17)  
ax2.set_xlabel(xlabel="时间", fontsize=17)  
plt.xticks(fontsize=15); plt.yticks(fontsize=15)  
fig.tight_layout(); plt.savefig(fname='fig/6_2.png')  

carb_diff2 = carb_diff.diff(12)  

fig = plt.figure(figsize=(12,4), dpi=150)  
ax = fig.add_subplot(111)  
ax.plot(carb_diff2, linestyle="-", color="blue")  
ax.set_ylabel(ylabel="季节差分",fontsize=17)  
ax.set_xlabel(xlabel="时间",fontsize=17)  
plt.xticks(fontsize=15); plt.yticks(fontsize=15)  
fig.tight_layout(); plt.savefig(fname='fig/6_3.png')  

fig = plt.figure(figsize=(12,4),dpi=150)  
ax1 = fig.add_subplot(121)  
ACF(carb_diff2.dropna(),lag=55)  
ax2 = fig.add_subplot(122)  
PACF(carb_diff2.dropna(),lag=55, xlabel='lag', fname='fig/6_4.png')  

mod = sm.tsa.statespace.SARIMAX(carb_df, order=(0,1,1), seasonal_order=(0,1,1,12))  
res = mod.fit()  
print(res.summary().tables[1])  

resid = res.resid[20:]  
resid_sd = (resid-np.mean(resid))/np.std(resid)  
lags = np.arange(1,40)  
Box_test = acorr_ljungbox(resid_sd,lags = lags, boxpierce=True, return_df=True)  
LB_p = Box_test.lb_pvalue  

fig = plt.figure(figsize=(12,9),dpi=150)  
ax1 = fig.add_subplot(311)  
ax1.plot(resid_sd);plt.title('Standardized Residuals',fontsize=17)  
plt.xticks(fontsize=15);plt.yticks(fontsize=15)  

ax2 = fig.add_subplot(323)  
ACF(resid);plt.title('ACF of Residuals',fontsize=17)  

ax3 = fig.add_subplot(324)  
sm.qqplot(resid_sd, fit=True, line="s",ax=ax3)  
plt.title('Normal Q-Q Plot of Std Residuals',fontsize=17)  
plt.xticks(fontsize=15);plt.yticks(fontsize=15)  

ax4 = fig.add_subplot(313)  
ax4.scatter(lags,LB_p,color="blue")  
plt.xlabel(xlabel="lag",fontsize=17)  
plt.ylabel(ylabel="P values",fontsize=17)  
plt.title('P values for Ljung-Box statistic',fontsize=17)  
plt.xticks(fontsize=15);plt.yticks(fontsize=15)  

fig.tight_layout()  
plt.savefig(fname='fig/6_5.png')  

### 例6.2

carb_data = np.loadtxt("carbondioxide.txt")  
index = pd.date_range(start="1966-01", end="1991-01", freq="M")  
carb_full = pd.Series(carb_data,index=index)  

carb_drop = carb_data[0:290]  
index = pd.date_range(start="1966-01", end="1990-03", freq="M")  
carb_df = pd.Series(carb_drop, index=index)  

res = sm.tsa.statespace.SARIMAX(carb_df, order=(0,1,1), seasonal_order=(0,1,1,12)).fit()  
print(res.summary().tables[1])  

carb_pred = res.get_prediction(start=290,end=299)  
confint1 = carb_pred.conf_int(alpha=0.20)  
confint2 = carb_pred.conf_int(alpha=0.05)  

confint = pd.concat([carb_pred.predicted_mean, confint1, confint2], axis=1, ignore_index=False)  
print(confint)  

fig = plt.figure(figsize=(12,4),dpi=150)  
ax1 = fig.add_subplot(121)  
ax1.plot(carb_df,linestyle="-",linewidth=0.8, color="green")  
ax1.plot(carb_pred.predicted_mean, linestyle=":", color="blue")  
ax1.set_ylabel(ylabel="二氧化碳排放量",fontsize=17)  
ax1.fill_between(confint1.index,confint1.iloc[:,0],confint1.iloc[:,1],color='k',alpha=.2)  
ax1.fill_between(confint2.index,confint2.iloc[:,0],confint2.iloc[:,1],color='k',alpha=.2)  
ax1.set_xlabel(xlabel="时间",fontsize=17)  
plt.xticks(fontsize=15); plt.yticks(fontsize=15)  
ax2 = fig.add_subplot(122)  
ax2.plot(carb_full,linestyle="-",linewidth=0.8, color="green")  
ax2.plot(carb_pred.predicted_mean,linestyle=":", color="blue")  
plt.legend(loc=2,labels=['实际值','预测值'],fontsize=13)  
ax2.set_xlabel(xlabel="时间",fontsize=17)  
plt.xticks(fontsize=15); plt.yticks(fontsize=15)  
fig.tight_layout(); plt.savefig(fname='fig/6_6.png')  